In [111]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re, time,csv
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [112]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
url = 'https://www.rottentomatoes.com/m/exodus_gods_and_kings/reviews'

In [126]:
"""
Extracts all the story links from the current page and writes to the specified writer
"""

def get_links_from_page(writer: csv.writer):
    
    #get all story divs in the current page
    reviews=driver.find_elements(by=By.CSS_SELECTOR,value='div[class="review-row"]')
    for review in reviews: # for each review div
        
        reviewer_name, review_date, review_text, review_polarity='NA','NA','NA','NA'
       
        try:
            reviewer_name=review.find_element(by=By.CSS_SELECTOR,value='a[class="display-name"]').text
        except NoSuchElementException as e: # reviewer name or link could not be found
            print('could not extract reviewer name')
            
        try:
            review_date=review.find_element(by=By.CSS_SELECTOR,value='p[class="original-score-and-url"]').text.strip().split('|')[-1].strip()
        except NoSuchElementException as e: # date or link could not be found
            print('could not extract review date')
    
            
        try: # try to get the review text
            review_text=review.find_element(by=By.CSS_SELECTOR,value='p[class="review-text"]').text
        except NoSuchElementException as e:
            print('could not extract review text')
            
        try: # try to get the review polarity
            review_polarity=review.find_element(by=By.CSS_SELECTOR,value='score-icon-critic').get_attribute('state')
        except NoSuchElementException as e:
            print('could not extract review polarity')
            
        #write a new row for this review
        writer.writerow([reviewer_name, review_date, review_text, review_polarity])


In [122]:
def scrape(driver, url):
    delay = 2
    #create a new csv writer for the story links
    fw = open('output.csv','w')

    writer=csv.writer(fw,lineterminator='\n')

    writer.writerow(['reviewer_name', 'review_date', 'review_text', 'review_polarity'])

    driver.get(url) # visit the page

    input() # wait until we can click on the cookie overlay

    page_cnt=1 # keep track of page count

    while True: # keep going until there are no more pages

        print('page',page_cnt, ' done') # print current page count

        page_cnt+=1 # increment 
        
        #extract and write the links from the current page
        get_links_from_page(writer)
        
        #wait until the next button appears
        wait = WebDriverWait(driver, delay)
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')))
        except:
            break

        #click the next button and wait 
        #next_button.click()
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(delay)
  
    fw.close()

In [123]:
scrape(driver, url)


page 1  done
page 2  done
page 3  done
page 4  done
page 5  done
page 6  done
page 7  done
page 8  done
page 9  done
page 10  done
page 11  done


In [124]:
import pandas as pd
df = pd.read_csv('output.csv')

In [125]:
df

,reviewer_name,review_date,review_text,review_polarity
0,Brian Eggert,"Jul 16, 2022",The most essential element lacking from the pr...,rotten
1,Kip Mooney,"Aug 10, 2021",How so many talented people came together here...,rotten
2,Richard Crouse,"Feb 2, 2021",Scott's film eschews all the Hollywood glam of...,rotten
3,Mike Massie,"Dec 4, 2020",Attempts to tell a famous tale with a rarely e...,fresh
4,Tony Asankomah,"Nov 20, 2020",It was very much pleasurable seeing all those ...,fresh
...,...,...,...,...
206,Stephen Farber,"Nov 29, 2014",No movie with such a limp ending can be fully ...,fresh
207,Justin Chang,"Nov 29, 2014",The result feels less like a straightforward r...,fresh
208,Alonso Duralde,"Nov 29, 2014",You'll leave humming the plagues.,rotten
209,Andy Lea,"Nov 29, 2014",The plot may be predictable but Scott's film i...,fresh


### Parkash 